In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digitaledu-competition/train.csv
/kaggle/input/digitaledu-competition/test.csv


In [29]:
data_test = pd.read_csv("/kaggle/input/digitaledu-competition/test.csv")
data_train= pd.read_csv("/kaggle/input/digitaledu-competition/train.csv")

In [10]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8193 entries, 0 to 8192
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                8193 non-null   int64  
 1   sex               8193 non-null   int64  
 2   bdate             7025 non-null   object 
 3   has_photo         8193 non-null   int64  
 4   has_mobile        8193 non-null   float64
 5   followers_count   8193 non-null   float64
 6   graduation        8193 non-null   float64
 7   education_form    7585 non-null   object 
 8   relation          8193 non-null   float64
 9   education_status  8193 non-null   object 
 10  langs             8193 non-null   object 
 11  life_main         8193 non-null   object 
 12  people_main       8193 non-null   object 
 13  city              7611 non-null   object 
 14  last_seen         8193 non-null   object 
 15  occupation_type   8000 non-null   object 
 16  occupation_name   8000 non-null   object 


In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.compose import ColumnTransformer

# Example: Fill missing values
data_train['education_form'].fillna(data_train['education_form'].mode()[0], inplace=True)
data_test['education_form'].fillna(data_test['education_form'].mode()[0], inplace=True)

# Convert 'bdate' and compute 'age'
data_train['bdate'] = pd.to_datetime(data_train['bdate'], format='%d.%m.%Y', errors='coerce')
ref_date = pd.to_datetime("01-01-2022", format='%d-%m-%Y')
data_train['age'] = (ref_date - data_train['bdate']).dt.days / 365.25
median_age = data_train['age'].median()
data_train['age'].fillna(median_age, inplace=True)

data_test['bdate'] = pd.to_datetime(data_test['bdate'], format='%d.%m.%Y', errors='coerce')
data_test['age'] = (ref_date - data_test['bdate']).dt.days / 365.25
data_test['age'].fillna(data_test['age'].median(), inplace=True)

cols_to_drop1 = ['id','bdate', 'last_seen', 'career_start', 'career_end', 'occupation_name']
cols_to_drop = [ 'bdate', 'last_seen', 'career_start', 'career_end', 'occupation_name']

data_train = data_train.drop(columns=cols_to_drop1)
data_test  = data_test.drop(columns=[col for col in cols_to_drop if col in data_test.columns])

# --- Prepare Features and Target ---
Y_train = data_train['result']
X_train = data_train.drop('result', axis=1)

# Convert numeric columns to float32
numeric_cols = X_train.select_dtypes(exclude=['object']).columns.tolist()
for col in numeric_cols:
    X_train[col] = pd.to_numeric(X_train[col], errors='coerce').fillna(0).astype(np.float32)
    data_test[col] = pd.to_numeric(data_test[col], errors='coerce').fillna(0).astype(np.float32)

# Identify categorical columns
categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()

# --- Preprocessing Pipeline ---
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(sparse=False, dtype=np.float32, handle_unknown='ignore'), categorical_cols),
        ('num', 'passthrough', numeric_cols)
    ]
)

# Transform the training and test features
X_train_processed = preprocessor.fit_transform(X_train).astype(np.float32)
X_test_processed  = preprocessor.transform(data_test).astype(np.float32)

# --- Modeling Pipeline ---
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.neighbors import KNeighborsClassifier

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('feature_selection', SelectKBest(score_func=f_classif, k=10)),  # Adjust k as needed.
    ('knn', KNeighborsClassifier(n_neighbors=10))
])

pipeline.fit(X_train_processed, Y_train)
y_pred = pipeline.predict(X_test_processed)

# --- Create and Save Results ---
results = pd.DataFrame({
    'id': data_test.get('id', range(len(y_pred))),  # Use the 'id' column if available
    'result': y_pred
})
print("\nPredictions:")
print(results.head())


/tmp/ipykernel_24/1171737769.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_train['education_form'].fillna(data_train['education_form'].mode()[0], inplace=True)
/tmp/ipykernel_24/1171737769.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[


Predictions:
      id  result
0  57927       0
1  30238       1
2  97835       1
3  85873       0
4  67939       0


In [31]:
results.to_csv('submisssions.csv', index=False)
